# Slicing XZ and ZY

In [1]:
import os
from os.path import join as opj
import gc
import glob

import torch
import numpy as np
import cv2
import tqdm
import pandas as pd
from loguru import logger

import utils

TESTING = False

SLICING_DATA = ['kidney_1_dense', 'kidney_3_dense', 'kidney_1_voi']

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/tamdiep/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_volume(dataset):
    path = os.path.join(dataset, "labels", "*.tif")
    dataset = sorted(glob.glob(path))
    volume = None
    target = None

    for z, path in enumerate(tqdm.tqdm(dataset)):
        label = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
        label = np.array(label,dtype=np.uint8)
        if target is None:
            target = np.zeros((len(dataset), *label.shape[-2:]), dtype=np.uint8)
        if volume is None:
            volume = np.zeros((len(dataset), *label.shape[-2:]), dtype=np.uint16)
        target[z] = label
        
        path = path.replace("/labels/", "/images/")
        path = path.replace("kidney_3_dense", "kidney_3_sparse")
        image = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
        image = np.array(image, dtype=np.uint16)
        volume[z] = image
    return volume, target


def save_volume(volume, target, dataset):
    if not os.path.exists(dataset): os.mkdir(dataset)
    if not os.path.exists(opj(dataset, "labels")): os.mkdir(opj(dataset, "labels"))
    if not os.path.exists(opj(dataset, "images")): os.mkdir(opj(dataset, "images"))
    logger.info('saving {} images of dataset {}'.format(dataset, volume.shape[0]))
    for z in tqdm.tqdm(range(volume.shape[0])):       
        cv2.imwrite(opj(dataset, "images",f"{z:04d}.tif"), volume[z])
        cv2.imwrite(opj(dataset, "labels",f"{z:04d}.tif"), target[z])

In [3]:
for folder in SLICING_DATA:
    dataset = opj('blood-vessel-segmentation/train', folder)
    dataset_xz = dataset + "_xz"
    dataset_zy = dataset + "_zy"
    skip_xz = os.path.exists(dataset_xz) 
    skip_zy = os.path.exists(dataset_zy)
    if not skip_xz or not skip_zy:
        logger.info("load {}".format(dataset))
        volume, target = load_volume(dataset)
    if not skip_xz:
        logger.info("save {}".format(dataset_xz))
        volume_xz, target_xz = volume.transpose((1, 2, 0)), target.transpose((1, 2, 0))
        if not TESTING: save_volume(volume_xz, target_xz, dataset_xz)
    else:
        logger.info("skipping {}".format(dataset_xz))
    if not skip_zy:
        logger.info("save {}".format(dataset_zy))
        volume_zy, target_zy = volume.transpose(2,0,1), target.transpose(2,0,1)
        if not TESTING: save_volume(volume_zy, target_zy, dataset_zy)
    else:
        logger.info("skipping {}".format(dataset_zy))

2023-12-04 12:59:17.733 | INFO     | __main__:<module>:15 - skipping blood-vessel-segmentation/train/kidney_1_dense_xz
2023-12-04 12:59:17.734 | INFO     | __main__:<module>:21 - skipping blood-vessel-segmentation/train/kidney_1_dense_zy
2023-12-04 12:59:17.734 | INFO     | __main__:<module>:15 - skipping blood-vessel-segmentation/train/kidney_3_dense_xz
2023-12-04 12:59:17.734 | INFO     | __main__:<module>:21 - skipping blood-vessel-segmentation/train/kidney_3_dense_zy
2023-12-04 12:59:17.735 | INFO     | __main__:<module>:15 - skipping blood-vessel-segmentation/train/kidney_1_voi_xz
2023-12-04 12:59:17.735 | INFO     | __main__:<module>:21 - skipping blood-vessel-segmentation/train/kidney_1_voi_zy


In [4]:
gt_df = pd.read_csv('gt.csv', dtype={'slice': 'string'})

In [5]:
print(gt_df['group'].unique())

['kidney_1_dense' 'kidney_1_voi' 'kidney_2' 'kidney_3_dense'
 'kidney_3_sparse']


In [6]:
print(gt_df.iloc[0])

id                                        kidney_1_dense_0000
rle                                                       1 0
group                                          kidney_1_dense
slice                                                    0000
height                                                   1303
width                                                     912
img_path    blood-vessel-segmentation/train/kidney_1_dense...
msk_path    blood-vessel-segmentation/train/kidney_1_dense...
Name: 0, dtype: object


In [7]:
gt_df.head()

,id,rle,group,slice,height,width,img_path,msk_path
0,kidney_1_dense_0000,1 0,kidney_1_dense,0000,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
1,kidney_1_dense_0001,1 0,kidney_1_dense,0001,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
2,kidney_1_dense_0002,1 0,kidney_1_dense,0002,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
3,kidney_1_dense_0003,1 0,kidney_1_dense,0003,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
4,kidney_1_dense_0004,1 0,kidney_1_dense,0004,1303,912,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...


In [8]:
# inline works with torch tensors
# import inline

# Visualize the volume
# t_xy = torch.from_numpy((target.argmax(0) / target.shape[0]).astype(np.float32))
# t_xy[-1] = 1
# inline.plot(t_xy, "XY")

In [9]:
# Plot single slice
# inline.plot(torch.from_numpy(volume[500]/65535).float())

# Plot 32 slices for XZ and ZY
# inline.plot(torch.from_numpy(volume_xz[500:532]/65535).float())
# inline.plot(torch.from_numpy(volume_zy[500:532]/65535).float())

In [10]:
list_data = []
for folder in SLICING_DATA:
    dataset = opj('blood-vessel-segmentation/train', folder)
    dataset_xz = dataset + "_xz"
    dataset_zy = dataset + "_zy"
    subdata = [dataset_xz, dataset_zy]
    for data_ in subdata:
        logger.info('infering: {}'.format(data_))
        img_folder = opj(data_, 'images')
        # label_folder = opj(data, 'labels')
        for image in tqdm.tqdm(os.listdir(img_folder)):
            if image.split('.')[-1] != 'tif':
                continue
            path = opj(img_folder, image)
            image = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
            path_sep = path.split('/')[-1]
            id = '{}_{}'.format(path.split('/')[-3], path_sep.split('.')[0])
            slice = path_sep.split('.')[0]
            try:
                height, width = image.shape
            except Exception as e:
                logger.error(e)
                logger.info(path)
                exit()
            msk_path = path.replace('images', 'labels')
            labels_image = cv2.imread(msk_path, cv2.IMREAD_ANYDEPTH)
            rle = utils.rle_encode(np.array(labels_image, dtype=np.uint8))
            group = path.split('/')[-3]
            row_data = [id, rle, group, slice, height, width, path, msk_path]
            list_data.append(row_data)
        
    

2023-12-04 12:59:22.137 | INFO     | __main__:<module>:8 - infering: blood-vessel-segmentation/train/kidney_1_dense_xz
100%|██████████████████████████████████████████████████████████████████████████| 1304/1304 [00:38<00:00, 33.77it/s]
2023-12-04 13:00:00.756 | INFO     | __main__:<module>:8 - infering: blood-vessel-segmentation/train/kidney_1_dense_zy
100%|████████████████████████████████████████████████████████████████████████████| 912/912 [00:38<00:00, 23.92it/s]
2023-12-04 13:00:38.877 | INFO     | __main__:<module>:8 - infering: blood-vessel-segmentation/train/kidney_3_dense_xz
100%|██████████████████████████████████████████████████████████████████████████| 1706/1706 [00:19<00:00, 87.57it/s]
2023-12-04 13:00:58.361 | INFO     | __main__:<module>:8 - infering: blood-vessel-segmentation/train/kidney_3_dense_zy
100%|██████████████████████████████████████████████████████████████████████████| 1510/1510 [00:19<00:00, 76.17it/s]
2023-12-04 13:01:18.186 | INFO     | __main__:<module>:8 - i

In [11]:
print(len(list_data))

9287


In [14]:
df_additional = pd.DataFrame(list_data, columns=['id', 'rle', 'group', 'slice', 'height', 'width', 'img_path', 'msk_path']) 

In [15]:
df_additional.head()

,id,rle,group,slice,height,width,img_path,msk_path
0,kidney_1_dense_xz_0937,185461 1 187738 3 190015 4 192294 1 203769 1 2...,kidney_1_dense_xz,0937,912,2279,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
1,kidney_1_dense_xz_0545,226477 2 228756 2 244872 1 247151 1 253834 1 2...,kidney_1_dense_xz,0545,912,2279,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
2,kidney_1_dense_xz_0124,1028397 4 1030678 2 1055733 5 1058014 2 106049...,kidney_1_dense_xz,0124,912,2279,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
3,kidney_1_dense_xz_0683,226490 1 228768 2 231047 2 233326 2 235604 2 2...,kidney_1_dense_xz,0683,912,2279,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...
4,kidney_1_dense_xz_0652,201402 2 203682 1 228825 1 231104 1 242758 1 2...,kidney_1_dense_xz,0652,912,2279,blood-vessel-segmentation/train/kidney_1_dense...,blood-vessel-segmentation/train/kidney_1_dense...


In [16]:
df_additional.tail()

,id,rle,group,slice,height,width,img_path,msk_path
9282,kidney_1_voi_zy_0305,29273 1 36971 10 38900 7 40821 14 42748 13 446...,kidney_1_voi_zy,0305,1397,1928,blood-vessel-segmentation/train/kidney_1_voi_z...,blood-vessel-segmentation/train/kidney_1_voi_z...
9283,kidney_1_voi_zy_1545,674477 8 676403 13 678330 14 680258 16 682185 ...,kidney_1_voi_zy,1545,1397,1928,blood-vessel-segmentation/train/kidney_1_voi_z...,blood-vessel-segmentation/train/kidney_1_voi_z...
9284,kidney_1_voi_zy_1683,1052 61 2981 61 4909 61 6837 62 8764 64 10693 ...,kidney_1_voi_zy,1683,1397,1928,blood-vessel-segmentation/train/kidney_1_voi_z...,blood-vessel-segmentation/train/kidney_1_voi_z...
9285,kidney_1_voi_zy_1124,1794 24 3721 25 5649 26 7577 26 9505 26 11433 ...,kidney_1_voi_zy,1124,1397,1928,blood-vessel-segmentation/train/kidney_1_voi_z...,blood-vessel-segmentation/train/kidney_1_voi_z...
9286,kidney_1_voi_zy_0764,651 89 2581 88 4509 87 6435 90 8362 90 10290 9...,kidney_1_voi_zy,0764,1397,1928,blood-vessel-segmentation/train/kidney_1_voi_z...,blood-vessel-segmentation/train/kidney_1_voi_z...


In [17]:
print(len(gt_df))

7429


In [19]:
df = pd.concat([gt_df, df_additional], ignore_index=True)

In [20]:
print(len(df))

16716


In [21]:
df.to_csv('gt_all.csv', index=False)